# Definition of a simple n-gram Model

In [1]:
# packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils
import torch.utils.data
from torch.utils.data import DataLoader

import pandas as pd
import numpy as np

import os

loading the data

In [24]:
path = '/home/lorenz/Documents/Mathematik/24 FS/Semester_Paper_DCML/data/ABC/harmonies/n10op74_02.harmonies.tsv'
df_all = pd.read_csv(path)

# define new dataframe df with only the 'numeral' column
df = df_all['numeral']
df_ = pd.factorize(df)
print(len(df_[1]))

ParserError: Error tokenizing data. C error: Expected 3 fields in line 4, saw 4


Markov chain model

In [25]:
class MarkovChain:
    def __init__(self, df):
        self.df = df
        self.df_ = pd.factorize(df)
        self.trans_mat = np.zeros((len(df_[1]), len(df_[1])))
    
    def clean_data(self):
        # if df_ contains a None value, remove it and the corresponding entry in df
        if None in self.df_[1]:
            ind = np.argwhere(self.df_[1] is None)
            self.df = np.delete(self.df, ind)
            self.df_ = pd.factorize(self.df)

    # calculate the transition matrix
    def markov_transition_matrix(self):
        trans_mat = np.zeros((len(self.df_[1]), len(self.df_[1])))

        # count of transitions
        for ch1 in self.df_[1]:
            inds = np.argwhere(self.df == ch1).flatten()
            # if first entry = 0, remove it 
            if inds[0] == 0:
                inds = np.delete(inds, 0)
            for ch2 in self.df_[1]:
                before = sum([chd == ch2 for chd in self.df[inds-1]])
                trans_mat[self.df_[1] == ch1, self.df_[1] == ch2] = before/len(self.df[inds])
        self.trans_mat = trans_mat
        return trans_mat

    def transform_chords_to_vectors(self, ch):
        assert(ch in self.df_[1])
        ind =  np.argwhere(self.df_[1] == ch)[0]
        vec = np.zeros(len(self.df_[1]))
        vec[ind] = 1
        return vec

    def fit(self):
        self.clean_data()
        self.trans_mat = self.markov_transition_matrix()
        return self

    def step(self, ch):
        # print('step form ', ch)
        vec = self.transform_chords_to_vectors(ch)
        out = np.dot(self.trans_mat.T, vec)
        return np.random.choice(self.df_[1], 1, p=out)[0]

    def predict(self, ch, n=3, verbose=False, start_at_current=False):
        seq = []
        if start_at_current:
            seq.append(ch)

        for i in range(n):
            ch = self.step(ch)
            seq.append(ch)
            if verbose:
                print(ch)
        return seq

fit the model

In [26]:
markov = MarkovChain(df).fit()
# markov.predict('I', n=36, verbose=True)

sonification

In [29]:
from midiutil import MIDIFile
from mingus.core import progressions

# transform markov output to readable chords with mignus
def chords_to_notes(progression, key='F'):
    return progressions.to_chords(progression, key)

def swap_accidentals(note):
    if note == 'Db':
        return 'C#'
    if note == 'D#':
        return 'Eb'
    if note == 'E#':
        return 'F'
    if note == 'Gb':
        return 'F#'
    if note == 'G#':
        return 'Ab'
    if note == 'A#':
        return 'Bb'
    if note == 'B#':
        return 'C'
    if note == 'Cb':
        return 'B'
    if note == 'Bbb':
        return 'A'
    return note

def note_to_number(note: str, octave: int) -> int:
    note = swap_accidentals(note)
    NOTES = ['C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'Ab', 'A', 'Bb', 'B']
    OCTAVES = list(range(11))
    NOTES_IN_OCTAVE = len(NOTES)
    note = swap_accidentals(note)
    if note not in NOTES:
        print('error: ', note)
    assert note in NOTES
    assert octave in OCTAVES

    note = NOTES.index(note)
    note += (NOTES_IN_OCTAVE * octave)
    return note

def midi_to_wav(midi_file):
    soundfont = '~/Documents/Musik/Soundfont/GeneralUser GS 1.471/GeneralUser GS v1.471.sf2'
    # convert midi to wav using fluidsynth
    wav_file = midi_file.replace('.mid', '.wav')
    os.system(f'fluidsynth -ni {soundfont} {midi_file} -F {wav_file} -r 44100')
    os.remove(midi_file)

# create a midi file
def create_midi_file(chord_progression, filename='output.mid',
                     duration=2, vol=100, octave=4):
    # create a midi file
    midi = MIDIFile(1)
    midi.addTempo(0, 0, 120)

    # change chord symbols to notes
    note_progression = chords_to_notes(chord_progression)
    print(type(note_progression))

    # add chords
    time = 0
    for chord in note_progression:
        for pitch in chord:
            pitch = note_to_number(pitch, octave)
            midi.addNote(0, 0, pitch, time, duration , vol)
        time += duration

    # write to file
    with open(filename, 'wb') as f:
        midi.writeFile(f)

    # convert to wav
    midi_to_wav(filename)
    print('midi file ' + filename + ' created')

In [28]:
# test
start = 'I'
length = 32
dur = 2
chords = markov.predict(start, n=length, start_at_current=True)

create_midi_file(chords, filename='markov_chain_output.mid', duration=dur)

org_chords = df.values[:length]
create_midi_file(org_chords, filename='original_output.mid', duration=dur)

<class 'list'>


Parameter '/home/lorenz/Documents/Musik/Soundfont/GeneralUser' not a SoundFont or MIDI file or error occurred identifying it.
Parameter 'GS' not a SoundFont or MIDI file or error occurred identifying it.
Parameter '1.471/GeneralUser' not a SoundFont or MIDI file or error occurred identifying it.
Parameter 'GS' not a SoundFont or MIDI file or error occurred identifying it.
Parameter 'v1.471.sf2' not a SoundFont or MIDI file or error occurred identifying it.


FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file 'markov_chain_output.wav'..
midi file markov_chain_output.mid created
<class 'list'>


Parameter '/home/lorenz/Documents/Musik/Soundfont/GeneralUser' not a SoundFont or MIDI file or error occurred identifying it.
Parameter 'GS' not a SoundFont or MIDI file or error occurred identifying it.
Parameter '1.471/GeneralUser' not a SoundFont or MIDI file or error occurred identifying it.
Parameter 'GS' not a SoundFont or MIDI file or error occurred identifying it.
Parameter 'v1.471.sf2' not a SoundFont or MIDI file or error occurred identifying it.


FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file 'original_output.wav'..
midi file original_output.mid created
